# Final project
# Yuan Liu 
### Embedding matrices by Deepwalk, LINE, node2vec were prepared in advance

In [1]:
import scipy.io as sio
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
matFile = sio.loadmat("../Homo_sapiens.mat")
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import spectral_embedding
from sklearn.decomposition import PCA
from sklearn.utils import shuffle as skshuffle
from sklearn.metrics import f1_score
from random import shuffle
import pandas as pd

In [2]:
matFile

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Nov 12 18:54:12 2015',
 '__version__': '1.0',
 '__globals__': [],
 'group': <3890x50 sparse matrix of type '<class 'numpy.float64'>'
 	with 6640 stored elements in Compressed Sparse Column format>,
 'network': <3890x3890 sparse matrix of type '<class 'numpy.float64'>'
 	with 76584 stored elements in Compressed Sparse Column format>}

In [3]:
matFile["group"].todense().shape

(3890, 50)

## Prepare all features used in Logistic regression

#### Seven basic features

In [4]:
G = nx.from_numpy_matrix(matFile["network"].todense()) # convert sprase to dense
degreeCentrality = nx.degree_centrality(G)
eigenCentrality = nx.eigenvector_centrality(G)
closenessCentrality = nx.closeness_centrality(G)
harmonicCentrality = nx.harmonic_centrality(G)
betweenCentrality = nx.betweenness_centrality(G)
pageRank = nx.pagerank(G)
clusteringCoefficient = nx.clustering(G)

In [5]:
#The order: degreeCen, Eigencen, CloseCen, HarmoCen, BetweenCen, pageRank,ClustCoeff
sevenFeature = np.vstack((list(degreeCentrality.values()),list(eigenCentrality.values()),
                          list(closenessCentrality.values()),list(harmonicCentrality.values()),
                          list(betweenCentrality.values()), list(pageRank.values()),
                          list(clusteringCoefficient.values()) )).T

In [6]:
sevenFeature[[1,2],]

array([[3.34276164e-03, 1.29503054e-02, 3.45445358e-01, 1.41263333e+03,
        3.41774580e-05, 1.57948498e-04, 4.35897436e-01],
       [5.14271021e-04, 1.45763538e-04, 2.61494295e-01, 1.04835000e+03,
        1.33089280e-05, 7.83700367e-05, 0.00000000e+00]])

#### Deepwalk LINE node2vec (7 + 128)

In [7]:
ls

deepwalk128.emb  Final_Project_Yuan_Liu.ipynb  line7.emb        node2vec7.emb
deepwalk7.emb    line128.emb                   node2vec128.emb


In [8]:
deepWalk7 = np.loadtxt("deepwalk7.emb",delimiter = " ",skiprows = 1,usecols=range(1,8))
deepWalk128 = np.loadtxt("deepwalk128.emb",delimiter = " ",skiprows = 1,usecols=range(1,129))
line7 = np.loadtxt("line7.emb",delimiter = " ",skiprows = 1,usecols=range(1,8))
line128 = np.loadtxt("line128.emb",delimiter = " ",skiprows = 1,usecols=range(1,129))
node2vec7 = np.loadtxt("node2vec7.emb",delimiter = " ",skiprows = 1,usecols=range(1,8))
node2vec128 = np.loadtxt("node2vec128.emb",delimiter = " ",skiprows = 1,usecols=range(1,129))

#### Spectral and Modularity (7 + 500)

In [9]:
spectral7 = spectral_embedding(matFile["network"],n_components=7)
spectral500 = spectral_embedding(matFile["network"],n_components=500)

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [10]:
graphDegree = np.array(list(dict(G.degree).values()))

modularityMatrix = matFile["network"].todense() - np.dot(graphDegree,graphDegree.T)/G.number_of_edges()/2

pca = PCA(n_components=7)
#pca.fit(modularityMatrix).explained_variance_ratio_
modularMatrixPC7 = pca.fit(modularityMatrix).components_.T

pca = PCA(n_components=500)
#pca.fit(modularityMatrix).explained_variance_ratio_
modularMatrixPC500 = pca.fit(modularityMatrix).components_.T

#### So far, we have the features below.
      1.sevenFeatures
      2.deepwalk7
      3.deepwalk128
      4.line7
      5.line128
      6.node2vec7
      7.node2vec128
      8.spectral7
      9.spectral500
      10.modularityMatrixPC7
      11.modularityMatrixPC500

## Start to test

In [15]:

featuresSet = [sevenFeature,deepWalk7,deepWalk128,line7,line128,node2vec7,node2vec128,spectral7,spectral500,
              modularMatrixPC7,modularMatrixPC500]
training_percents = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
microF1Holder = pd.DataFrame(columns=["Methods"])
microF1Holder["Methods"]=["SevenFeatures","Deepwalk-7","Deepwalk-128","LINE-7","LINE-128",
                         "node2vec-7","node2vec-128","spectral-7","spectral-500","modularity-7","modularity-500"]
macroF1Holder = pd.DataFrame(columns=["Methods"])
macroF1Holder["Methods"]=["SevenFeatures","Deepwalk-7","Deepwalk-128","LINE-7","LINE-128",
                         "node2vec-7","node2vec-128","spectral-7","spectral-500","modularity-7","modularity-500"]

In [16]:
totalInstance = 3890

for training_percent in training_percents:
    microF1Set, macroF1Set = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]), np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    for i in range(0,10): #shuffle 10 times
        microF1 = np.array([])
        macroF1 = np.array([])
        for features in featuresSet:
            f1_micro,f1_macro = 0, 0
            y_preds, y_trues = [], []
            for j in range(0,50): #iterate 50 labels
                labels = matFile["group"].todense()[:,j]
                allInstance =  list(range(0,totalInstance))
                shuffle(allInstance)# Random select instances
                trainingInstance = np.array(allInstance[0:int(training_percent*totalInstance)])
                testingInstance = np.setdiff1d(np.array(range(0,totalInstance)),trainingInstance)
                logReg = LogisticRegression(penalty="l2",solver='lbfgs',
                                    max_iter=100,verbose = 0).fit(features[trainingInstance,:],
                                    np.array(labels[trainingInstance]).ravel())
                y_preds.append(logReg.predict(features[testingInstance,]))
                y_trues.append(np.array(labels[testingInstance]).ravel())
                
            microF1 = np.append(microF1,[f1_score(np.asarray(y_trues),np.asarray(y_preds),average="micro")])
            macroF1 = np.append(macroF1,[f1_score(np.asarray(y_trues),np.asarray(y_preds),average="macro")])  
        microF1Set = microF1Set + microF1
        macriF1Set = macroF1Set + macroF1
    microF1Holder[training_percent] = microF1Set/10
    macroF1Holder[training_percent] = macroF1Set/10

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/ana

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/

...

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/ana

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/ana

/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abc/anaconda2/envs/py3envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [17]:
microF1Holder

,Methods,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,SevenFeatures,0.001380,0.002014,0.001590,0.001802,0.001883,0.002016,0.002846,0.001167
1,Deepwalk-7,0.000261,0.000340,0.000302,0.000241,0.000150,0.000295,0.000000,0.000000
2,Deepwalk-128,0.008066,0.007176,0.005362,0.004137,0.002576,0.001694,0.001529,0.000593
3,LINE-7,0.011566,0.010630,0.009423,0.010425,0.012268,0.012242,0.012592,0.010105
4,LINE-128,0.098769,0.102834,0.104430,0.106735,0.106126,0.107873,0.104764,0.098741
5,node2vec-7,0.000299,0.000043,0.000050,0.000000,0.000000,0.000000,0.000000,0.000000
6,node2vec-128,0.001078,0.001024,0.000497,0.000299,0.000000,0.000100,0.000000,0.000000
7,spectral-7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,spectral-500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,modularity-7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
macroF1Holder

,Methods,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,SevenFeatures,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Deepwalk-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Deepwalk-128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LINE-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LINE-128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,node2vec-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,node2vec-128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,spectral-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,spectral-500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,modularity-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
macroF1

array([0.        , 0.        , 0.00257069, 0.00616967, 0.06109683,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])